In [57]:
import json
import pandas as pd
import requests
import os
import xml.etree.ElementTree as ET
import xmltodict
import matplotlib.pyplot as plt

In [2]:
filepath = 'data/'
filelist = os.listdir(filepath)
# filelist
conf = open('config.json','r') 
config = json.load(conf)
api_key = config['API_key']
conf.close()
headers=  {'X-ELS-APIKey':api_key}

In [3]:
filelist

['carbon emissions.csv',
 'clean technology.csv',
 'clean-energy.csv',
 'energy efficient.csv',
 'geothermal.csv',
 'heat recovery.csv']

In [4]:
filename = 'geothermal.csv'

In [5]:
raw_data = pd.read_csv(filepath + filename,header=0)

In [50]:
raw_data.head()


,title,autor,doi,citedby_count,affilname,affiliation-city,affiliation-country,abstract
0,Ecological responses to recent climate change,Walther G.,10.1038/416389a,7097,Gottfried Wilhelm Leibniz Universität Hannover,Hannover,Germany,None
1,Composition of the Continental Crust,Rudnick R.L.,10.1016/B0-08-043751-6/03016-4,6337,"University of Maryland, College Park",College Park,United States,None
2,Convection in porous media,Nield D.A.,10.1007/978-1-4614-5541-7,6284,The University of Auckland,Auckland,New Zealand,None
3,Opportunities and challenges for a sustainable...,Chu S.,10.1038/nature11475,6268,United States Department of Energy,"Washington, D.C.",United States,None
4,"A review of the source, behaviour and distribu...",Smedley P.L.,10.1016/S0883-2927(02)00018-5,5747,British Geological Survey,Nottingham,United Kingdom,None


In [48]:
raw_data["abstract"] = None

In [52]:
for i in range(100):
    doi = raw_data['doi'][i]
    aburl = 'https://api.elsevier.com/content/article/doi/'+doi
    print(aburl)
    abstract = requests.get(aburl,headers = headers)
    try:
        d = xmltodict.parse(abstract.text)
        # print(d["full-text-retrieval-response"]['coredata']['dc:description'])
        abstract_text = d["full-text-retrieval-response"]['coredata']['dc:description']
        raw_data['abstract'][i] = abstract_text
        pass
    except:
        continue

https://api.elsevier.com/content/article/doi/10.1038/416389a
https://api.elsevier.com/content/article/doi/10.1016/B0-08-043751-6/03016-4


C:\Users\DylanTong\AppData\Local\Temp\ipykernel_29796\3210878497.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['abstract'][i] = abstract_text


https://api.elsevier.com/content/article/doi/10.1007/978-1-4614-5541-7
https://api.elsevier.com/content/article/doi/10.1038/nature11475
https://api.elsevier.com/content/article/doi/10.1016/S0883-2927(02)00018-5
https://api.elsevier.com/content/article/doi/10.1128/AEM.71.12.8228-8235.2005
https://api.elsevier.com/content/article/doi/10.1038/347662a0
https://api.elsevier.com/content/article/doi/10.1073/pnas.95.12.6578
https://api.elsevier.com/content/article/doi/10.1175/BAMS-D-14-00110.1
https://api.elsevier.com/content/article/doi/10.1093/petrology/36.4.891
https://api.elsevier.com/content/article/doi/10.2138/am-2000-0416
https://api.elsevier.com/content/article/doi/10.1016/j.jhazmat.2007.01.006
https://api.elsevier.com/content/article/doi/10.1016/S0009-2541(97)00150-2
https://api.elsevier.com/content/article/doi/10.1016/S0039-9140(02)00268-0
https://api.elsevier.com/content/article/doi/10.1785/0120000006
https://api.elsevier.com/content/article/doi/10.1017/CBO9780511536045
https://api.

In [86]:
countries = raw_data['affiliation-country'].value_counts()

countries = list(countries.index[0:11])
print(countries)

['United States', 'United Kingdom', 'China', 'France', 'Germany', 'Canada', 'Australia', 'Italy', 'Switzerland', 'Japan', 'New Zealand']


In [87]:
selected_data = raw_data[raw_data['affiliation-country'] in countries]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [91]:
raw_data[ if raw_data['affiliation-country']  in countries]

SyntaxError: invalid syntax (3710966229.py, line 1)